<a href="https://colab.research.google.com/github/joje3029/AI_Study/blob/main/2023_08_16_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80_%EC%A7%91%EA%B0%92%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [32]:
# 데이터 로드
house=pd.read_csv('/content/1553768847-housing.csv')

In [33]:
# EDA
## 결측치가 많거나 불필요한 컬럼은 걸러내 보자
## 상관관계로 필요한거 솎아내보자
## 이상치가 많아 보이는 컬럼은 체크해두자

# house : longitude(경도)(float),	latitude(위도)(float),	housing_median_age(주택 평균 연령)(int),	total_rooms(총 방)(int),	total_bedrooms(총 침실)(null)(float),	population(인구)(int),	households(가구)(int),	median_income(중간 소득)(float),	ocean_proximity(바다 근접)(object),	median_house_value(중간 집값)(int)
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  int64  
 3   total_rooms         20640 non-null  int64  
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  int64  
 6   households          20640 non-null  int64  
 7   median_income       20640 non-null  float64
 8   ocean_proximity     20640 non-null  object 
 9   median_house_value  20640 non-null  int64  
dtypes: float64(4), int64(5), object(1)
memory usage: 1.6+ MB


In [34]:
## 이상치 존재 여부 확인 -> IQR

Q1=house.quantile(0.25) #1분위 값
Q3=house.quantile(0.75) #3분위 값

IQR = Q3-Q1 #IQR

hightest = Q3+IQR*1.5 #상한선 계산
lowest = Q3-IQR*1.5 #하한선 계산



<ipython-input-34-dd13a18201ed>:3: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q1=house.quantile(0.25) #1분위 값
<ipython-input-34-dd13a18201ed>:4: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q3=house.quantile(0.75) #3분위 값


In [ ]:
## 위경도를 이용한 산점도 그리기
sns.scatterplot(x="longitude", y="latitude", data=house)
plt.title("위경도 산점도")
plt.show()

In [36]:
## 집값(median_house_value)과 다른 변수의 상관관계 확인 -> 히트맵 사용 : 타입 object 가 있으므로 카테고리값 변환/결측치 one-hot 인코딩 등을 해야함.
# sns.heatmap(data=house, annot=True, cmap="coolwarm", linewidths=.5) -> 여기서는 원핫 인코딩을 하는게 좋을듯

In [ ]:
## 위도(longitude) 경도(latitude)를 이용해 산점도로 위치와 집값의 관계 시각적으로 확인
sns.scatterplot(data=house, x='longitude', y='latitude', hue='median_house_value')

In [ ]:
# 데이터 전처리
## 결측치 처리
notnull_house=house.dropna()
notnull_house.info()

In [ ]:
## 이상치 처리
# notnull_house.describe()

# 수치형 데이터와 범주형 데이터 분리
# 수치형 데이터
int_house=notnull_house.drop(columns='ocean_proximity')

# 범주형 데이터
object_house=notnull_house['ocean_proximity']


In [ ]:
## 수치형 데이터
### 표준화
from sklearn.preprocessing import StandardScaler
scalar= StandardScaler()

standardised_data=scalar.fit_transform(int_house)
int_house2=pd.DataFrame(standardised_data, columns=int_house.columns).reset_index()
int_house2

In [ ]:
## 범주형 데이터
### 원핫인코딩
object_house2=pd.get_dummies(object_house).reset_index()
object_house2

In [ ]:
# 범주형이랑 수치형 합치기
new_house=pd.concat([int_house2,object_house2], axis=1)
new_house

In [77]:
# 학습 데이터와 정답 데이터 분리
train=new_house.drop(columns='median_house_value')

test=new_house['median_house_value']

from sklearn.model_selection import train_test_split

trd, tsd, trt, tst = train_test_split(train, test, train_size=0.8, random_state=42)
print(trd.shape, tsd.shape)

(16346, 15) (4087, 15)


In [81]:
# 학습
## 선형회귀를 이용하여 모델 학습 : from sklearn.linear_model import LinearRegression 이거 사용
from sklearn.linear_model import LinearRegression

lr=LinearRegression()

lr.fit(trd,trt)

LinearRegression()

In [82]:
# 평가
## 학습한 모델을 평가하기
lr.score(tsd, tst)

0.6536462120329367